In [1]:
import pandas as pd
import numpy as np

In [7]:
git clone https://github.com/j-jwalker/Star-Clocks.git

SyntaxError: invalid syntax (2970698083.py, line 1)

In [2]:
# Section 1: Import the star charts
# Message to user: put the excel file in the same folder as this code, then type the name of the file next to file_path
file_path = "NPdata.xlsx" # <-- Right here!

try:
    # Read the Excel file using pandas
    df = pd.read_excel(file_path)

    # Remove first 4 rows
    df = df.iloc[4:]

    # Reset the index of the rows so it starts at 0 again
    df.reset_index(drop=True, inplace=True)

    # Rename the columns
    new_columns = {'horizon is (?, ?)': 'Hour','Unnamed: 1': '-3','Unnamed: 2': '-2','Unnamed: 3': '-1','Unnamed: 4': '0',
                   'Unnamed: 5': '1','Unnamed: 6': '2','Unnamed: 7': '3'}
    df.rename(columns=new_columns, inplace=True)
    
    # Set option to display all rows
    pd.set_option('display.max_rows', None)

    #print(df)

except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [3]:
# Section 2: Convert data to general format
# 2a) Looks through the star charts, outputs a list of the stars in it and counts how many stars are represented

# Function to return unique values from a DataFrame excluding specified values
# For our data, unique values are individual stars
def get_unique_values(dataframe, exclude_values=None):
    unique_values = set()
    for column in dataframe.columns:
        if exclude_values:
            unique_values.update([value for value in dataframe[column].unique() 
                                  if pd.notna(value) and value not in exclude_values])
        else:
            unique_values.update(dataframe[column].unique())
    return unique_values

# Specify values to be excluded, ie star clock position columns -3 to 3, as well as hour markings 0 to 12
exclude_values = set(range(-3, 13)) 
# Excludes the table names in the top left corner
exclude_values.update(['Table 1', 'Table 2', 'Table 3', 'Table 4', 'Table 5', 'Table 6', 
                        'Table 7', 'Table 8', 'Table 9', 'Table 10', 'Table 11', 'Table 12', 
                        'Table 13', 'Table 14', 'Table 15', 'Table 16', 'Table 17', 'Table 18', 
                        'Table 19', 'Table 20', 'Table 21', 'Table 22', 'Table 23', 'Table 24','?'])

# Get unique values from the DataFrame excluding specified values, then print the values (stars)
unique_values = get_unique_values(df, exclude_values)
#print("List of stars:")
#print(unique_values)

# Count the number of values in the unique_values set, then print the count
unique_values_count = len(unique_values)
print("Number of stars:", unique_values_count)

# Beginning of new dataframe, filling a column with each star
gen_df = pd.DataFrame({'Stars': list(unique_values)})

#print(gen_df)

Number of stars: 47


In [4]:
# 2b) Finds the index (row) and column of every star in the chart. From the index, also finds the table and hour for each star.

# Initialize an empty list to store dataframes
dfs = []

# Iterate through each value in gen_df
for col_gen, row_gen in gen_df.items():
    for idx_gen, val_gen in row_gen.items():
        # Search for the value in df
        for idx_df, row_df in df.iterrows():
            for col_df, val_df in row_df.items():
                # Check if the value matches
                if val_df == val_gen:
                    # Calculate table number based on index
                    table_number = (idx_df + 15) // 15
                    # Calculate hour based on index and table number
                    Hour = idx_df - 15*(table_number - 1) - 1
                    # Create a temporary DataFrame with the match
                    match_df = pd.DataFrame({'Star': [val_df],
                                             'Index': [idx_df],
                                             'Position': [col_df],
                                             'Table #': [table_number],
                                             'Hour': [Hour]})
                    # Append the temporary DataFrame to the list
                    dfs.append(match_df)

# Concatenate all temporary DataFrames into one DataFrame
int_df = pd.concat(dfs, ignore_index=True)

#print(int_df)

In [5]:
# 2c) Converts intermediate calculation table int_df into the ideal table id_df

# Create id_df with appropriate column names
columns = []
for i in range(1, 25):
    columns.extend([f'H{i}', f'P{i}'])

id_df = pd.DataFrame(index=gen_df['Stars'], columns=columns)  # Set index to 'Stars'

# Iterate through int_df and fill id_df accordingly
for index, row in int_df.iterrows():
    star_name = row['Star']
    table_num = row['Table #']
    position = row['Position']
    hour = row['Hour']
    
    if star_name in id_df.index:
        id_df.at[star_name, f'P{table_num}'] = position
        id_df.at[star_name, f'H{table_num}'] = hour

# Fill remaining positions with NaNs
id_df.fillna(np.nan, inplace=True)

# Set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(id_df)

         H1   P1    H2   P2    H3   P3    H4   P4    H5   P5    H6   P6    H7  \
Stars                                                                           
R4      NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN   
R6      NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN   
E6      NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN   
E7      2.0    0   1.0    1   0.0    0   NaN  NaN   NaN  NaN   NaN  NaN   NaN   
R8      NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN   
E11     4.0    3   3.0    3   2.0    0   NaN  NaN   NaN  NaN   NaN  NaN   NaN   
L1      NaN  NaN  12.0   -3  11.0    1   NaN  NaN   NaN  NaN   NaN  NaN   NaN   
M1      NaN  NaN   NaN  NaN  12.0    0  11.0   -3  10.0    0   9.0    0   7.0   
R5      NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN   
Q3      NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN   
P       NaN  NaN   NaN  NaN 